<a href="https://colab.research.google.com/github/Bhavnicksm/ABSA-SentiHood/blob/main/SentiHood_BERT_failure_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import json
from collections import Counter

In [48]:
train_path = "/content/drive/MyDrive/SentiHood/data/sentihood-train.json"
with open(train_path, 'r') as file:
  train = json.loads(file.read())

dev_path = "/content/drive/MyDrive/SentiHood/data/sentihood-dev.json"
with open(dev_path, 'r') as file:
  dev = json.loads(file.read())

test_path = "/content/drive/MyDrive/SentiHood/data/sentihood-test.json"
with open(test_path, 'r') as file:
  test = json.loads(file.read())


print(len(train), len(dev), len(test))

2977 747 1491


In [16]:
sub_path = "/content/drive/MyDrive/SentiHood/models/roberta-large/submission.jsonl"

with open(sub_path, 'r') as file:
  sub = []
  for line in file:
    sub.append(json.loads(line))


print(len(sub))

1491


In [17]:
err = []

for ex in sub:
  if ex['opinions'] != ex['model_pred']:
    err.append(ex)

print(len(err))

367


In [44]:
right = []

for ex in sub:
  if ex['opinions'] == ex['model_pred']:
    right.append(ex)

print(len(right))

1124


In [21]:
from pprint import pprint

In [ ]:
pprint(err)

In [29]:
fp = []
fn = []
for example in  err:
  mp = example['model_pred']
  op = example['opinions']
  if len(mp) > len(op):
    fp.append((op, mp))
  else:
    fn.append((op,mp))

  # print(fp, fn)

print(len(fp), len(fn))

185 182


In [39]:
err_fp_counter = Counter()
err_fn_counter = Counter()
err_fp_asp_counter = Counter()
err_fn_asp_counter = Counter()

for example in err:
  mp_set = set()
  for mp in example['model_pred']:
    mp_set.add(tuple(mp.values()))
  # print(mp_set)
  op_set = set()
  for op in example['opinions']:
    op_set.add(tuple(op.values()))
  # print(op_set)

  if len(mp_set) > len(op_set):
    err_exs = list(mp_set.difference(op_set))
    err_fp_counter.update(err_exs)
    for err_ex in err_exs:
      err_fp_asp_counter.update([err_ex[0]])
  else:
    err_exs = list(op_set.difference(mp_set))
    err_fn_counter.update(err_exs)
    for err_ex in err_exs:
      err_fn_asp_counter.update([err_ex[1]])

In [42]:
print("Aspects predicted falsely most commonly")
pprint(err_fp_asp_counter.most_common())

Aspects predicted falsely most commonly
[('general', 76),
 ('price', 47),
 ('transit-location', 44),
 ('safety', 37),
 ('nightlife', 33),
 ('live', 31),
 ('green-nature', 31),
 ('dining', 26),
 ('quiet', 24),
 ('multicultural', 23),
 ('shopping', 23),
 ('touristy', 17)]


In [43]:
print("Aspects missed out in prediction most commonly")
pprint(err_fn_asp_counter.most_common())

Aspects missed out in prediction most commonly
[('general', 124),
 ('transit-location', 73),
 ('price', 67),
 ('safety', 37),
 ('live', 23),
 ('quiet', 20),
 ('nightlife', 15),
 ('green-nature', 13),
 ('touristy', 13),
 ('shopping', 12),
 ('multicultural', 11),
 ('dining', 9)]


In [46]:
right_asp_counter = Counter()
for example in right:
  for pred in example['model_pred']:
    right_asp_counter.update([pred['aspect']])

In [47]:
print("Aspects predicted right most commonly")
pprint(right_asp_counter.most_common())

Aspects predicted right most commonly
[('general', 400),
 ('price', 158),
 ('transit-location', 123),
 ('safety', 105),
 ('live', 67),
 ('nightlife', 58),
 ('shopping', 57),
 ('multicultural', 36),
 ('green-nature', 29),
 ('dining', 22),
 ('touristy', 16),
 ('quiet', 14)]
